In [1]:
import os                                               # for working with files
import numpy as np                                      # for numerical computationss
import pandas as pd                                     # for working with dataframes
import torch                                            # Pytorch module
import gc                                               # for garbage collection
import seaborn as sns
import matplotlib.pyplot as plt                         # for plotting informations on graph and images using tensors
import torch.nn as nn                                   # for creating  neural networks
from torch.utils.data import DataLoader                 # for dataloaders 
from PIL import Image                                   # for checking images
import torch.nn.functional as F                         # for functions for calculating loss
import torchvision.transforms as transforms             # for transforming images into tensors 
from torchvision.utils import make_grid                 # for data checking
from torchvision.datasets import ImageFolder            # for working with classes and images
import torchvision.models as models                     # for pretrained models
from torchsummary import summary                        # for getting the summary of our model
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score
from yellowbrick.style import set_palette

In [2]:
data_dir = "New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = data_dir + "/train"
valid_dir = data_dir + "/valid"
diseases = os.listdir(train_dir)

In [3]:
# datasets for validation and training
train = ImageFolder(train_dir, transform=transforms.ToTensor())
valid = ImageFolder(valid_dir, transform=transforms.ToTensor())

In [4]:
# Setting the seed value
random_seed = 7
torch.manual_seed(random_seed)

# setting the batch size
batch_size = 32

# DataLoaders for training and validation
train_dl = DataLoader(train, batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid, batch_size, num_workers=2, pin_memory=True)

In [5]:
# for moving data into GPU (if available)
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available:
        return torch.device("cuda")
    else:
        return torch.device("cpu")

# for moving data to device (CPU or GPU)
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

def clear_gpu_memory():
    torch.cuda.empty_cache()
    gc.collect()

# for loading in the device (GPU if available else CPU)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)
        
    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [6]:
device = get_default_device()

In [7]:
# Moving data into GPU
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)

In [8]:
# for calculating the accuracy
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

# base class for the model
def training_step(model, batch):
    images, labels = batch
    out = model(images)                       # Generate predictions
    loss = F.cross_entropy(out, labels)       # Calculate loss
    return loss

def validation_step(model, batch):
    images, labels = batch
    out = model(images)                        # Generate prediction
    loss = F.cross_entropy(out, labels)        # Calculate loss
    acc = accuracy(out, labels)                # Calculate accuracy

    _, preds = torch.max(out, dim=1)
    return {"val_loss": loss.detach(), "val_accuracy": acc, "preds": preds, "labels": labels}

def validation_epoch_end(outputs):
    batch_losses = [x["val_loss"] for x in outputs]
    batch_accuracy = [x["val_accuracy"] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()       # Combine loss  
    epoch_accuracy = torch.stack(batch_accuracy).mean()

    all_preds = torch.cat([x['preds'] for x in outputs], dim=0)
    all_labels = torch.cat([x['labels'] for x in outputs], dim=0)

    return {"val_loss": epoch_loss, "val_accuracy": epoch_accuracy, "preds": all_preds.cpu().numpy(), "labels": all_labels.cpu().numpy()}

def epoch_end(epoch, result):
    print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
        epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_accuracy']))

In [9]:
model = models.efficientnet_b0()

# changing the last layer
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, len(train.classes))

# moving the model to GPU
model = to_device(model, device)

# getting summary of the model
INPUT_SHAPE = (3, 256, 256)
print(summary(model.cuda(), (INPUT_SHAPE)))

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 3.81 GiB total capacity; 97.97 MiB already allocated; 448.00 KiB free; 110.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [validation_step(model, batch) for batch in val_loader]
    return validation_epoch_end(outputs)


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    

def fit_OneCycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0,
                grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # scheduler for one cycle learning rate
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))
    
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = training_step(model, batch)
            train_losses.append(loss)
            loss.backward()
            
            # gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
                
            optimizer.step()
            optimizer.zero_grad()
            
            # recording and updating learning rates
            lrs.append(get_lr(optimizer))
            sched.step()
            
    
        # validation
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        epoch_end(epoch, result)
        history.append(result)
        
    return history

In [ ]:
%%time
history = [evaluate(model, valid_dl)]

In [ ]:
epochs = 2
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [ ]:
%%time
history += fit_OneCycle(epochs, max_lr, model, train_dl, valid_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=1e-4, 
                             opt_func=opt_func)

In [ ]:
# saving the model
PATH = './saved_models/efficientnetb0-pre-image-net.pth'  
torch.save(model.state_dict(), PATH)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_accuracy'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'].cpu().numpy() for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');
    
def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs', []) for x in history])
    sns.lineplot(data=lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.');

In [ ]:
plot_accuracies(history)

In [ ]:
plot_losses(history)

In [ ]:
plot_lrs(history)

In [ ]:
def obtain_performance_metrics(ground_truths, predictions):
    # Performance Metrics and Confusion Matrix
    print("Classification Report: ")
    print()
    target_names = train.classes
    print(classification_report(ground_truths, predictions, target_names=target_names))

    print("Balanced accuracy score: ")
    print(balanced_accuracy_score(ground_truths, predictions))

    print("\nConfusion Matrix: ")
    print()
    plt.figure(figsize=(20,20))
    matrix = confusion_matrix(ground_truths, predictions)
    sns.heatmap(matrix, cmap="PuBu", annot=True, linewidths=0.5, fmt= 'd')
    plt.show()


In [ ]:
obtain_performance_metrics(history[2]['labels'], history[2]['preds'])